In [1]:
!pip install torchsummary

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.nn import Module
from torch.nn import Sequential
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import AvgPool2d
from torch.nn import BatchNorm2d
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Tanh
from torch.nn import ELU
from torch.nn import LogSoftmax
from torch.nn import Dropout
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import flatten

from sklearn.metrics import accuracy_score, f1_score
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
from torch.utils.data.sampler import SubsetRandomSampler
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchsummary import summary
import cv2
import glob
import pickle
import matplotlib.pyplot as plt


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


# Dataset Processing

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

folder_path="/kaggle/input/cifar10-python/cifar-10-batches-py"

file_paths = glob.glob(folder_path + '/data_batch_*')

dicts=[]
for file_path in file_paths:
    # Do something with the file, for example, print its path
    dicts.append(unpickle(file_path))

images=[]
allLabels=[]
for d in dicts:
    keys_list = list(d.keys())

    labels=d[keys_list[1]]
    img_data=d[keys_list[2]]

    red_channel = img_data[:, :1024].reshape(10000, 32, 32)
    green_channel = img_data[:, 1024:2048].reshape(10000, 32, 32)
    blue_channel = img_data[:, 2048:].reshape(10000, 32, 32)

    image_array = np.stack((red_channel, green_channel, blue_channel), axis=-1)
    images.append(image_array)
    allLabels.append(labels)



In [6]:
def CIFARDataset(images,labels):
    newPath = './CIFAR_DataSet'

    os.makedirs(newPath, exist_ok=True)

    uniquelabels=np.unique(labels)
    mp={}

    for label in uniquelabels:
        rdpath=newPath+"/"+str(label)

        os.makedirs(rdpath, exist_ok=True)

        mp[label]=1

    for i in range(len(labels)):
        for j in range(len(labels[i])):

            img=images[i][j]
            img=img.astype(np.uint8)
            lab=labels[i][j]

            wrtPath=newPath+'/'+ str(lab) +'/'+str(mp[lab]) + '.jpg'
            
            cv2.imwrite(wrtPath,img)
            mp[lab]+=1
        


In [5]:
def loadDataset():
    imgs = []
    labels = []
    label = -1
    rdir_path = '/kaggle/working/CIFAR_DataSet'

    dirs = [f for f in os.listdir(rdir_path) if os.path.isdir(os.path.join(rdir_path, f))]
    
    for directory in dirs:
        label = label + 1
        path = rdir_path+'/'+directory
        onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        
        for file in onlyfiles:
            imgPath = path+'/'+file
            imgs.append(imgPath)
            labels.append(label)
    # print(imgs[0:100],labels[0:100])
    return (imgs,labels)

In [7]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 170498071/170498071 [00:09<00:00, 17759775.72it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [8]:
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
train_loader

In [9]:
#Architecture 2
class CNN(Module):
  def __init__(self, in_channels, classes):
    super(CNN, self).__init__()

    self.classes = classes

    self.conv_block_1 = Sequential(
        Conv2d(in_channels = in_channels, out_channels = 64,
               kernel_size = (3, 3), stride = (1, 1), padding = (2, 2)),
        BatchNorm2d(64),
        ReLU(),
        Conv2d(in_channels = 64, out_channels = 128,
               kernel_size = (3, 3), stride = (1, 1), padding = (2, 2)),
        BatchNorm2d(128),
        ReLU(),
        Conv2d(in_channels = 128, out_channels = 128,
               kernel_size = (3, 3), stride = (1, 1), padding = (0, 0)),
        BatchNorm2d(128),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )

    self.conv_block_2 = Sequential(
        Conv2d(in_channels = 128, out_channels = 256,
               kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
        BatchNorm2d(256),
        ReLU(),
        Conv2d(in_channels = 256, out_channels = 256,
               kernel_size = (3, 3), stride = (2, 2), padding = (1, 1)),
        BatchNorm2d(256),
        ReLU(),
        MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
    )
 
    self.fc_block = Sequential(
        Linear(in_features = 4 * 4 * 256, out_features = 2048),
        ReLU(),
        Dropout(0.3),
        Linear(in_features = 2048, out_features = 512),
        ReLU(),
        Dropout(0.3),
        Linear(in_features = 512, out_features = self.classes)
    )
    

  def forward(self, x):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = flatten(x, 1)
    x = self.fc_block(x)
    
#     out = LogSoftmax(x)

    return x


##Architecture 3
# class CNN(Module):
#   def __init__(self, in_channels, classes,activation):
#     super(CNN, self).__init__()
#     self.activation = activation
#     self.classes = classes

#     self.conv_block_1 = Sequential(
#         Conv2d(in_channels = in_channels, out_channels = 64,
#                kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
#         BatchNorm2d(64),
#         self.activation(),
#         Conv2d(in_channels = 64, out_channels = 128,
#                kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
#         BatchNorm2d(128),
#         self.activation(),
#         MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
#     )

#     self.conv_block_2 = Sequential(
#         Conv2d(in_channels = 128, out_channels = 256,
#                kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
#         BatchNorm2d(256),
#         self.activation(),
#         Conv2d(in_channels = 256, out_channels = 256,
#                kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
#         BatchNorm2d(256),
#         self.activation(),
#         MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
#     )
    
#     self.conv_block_3 = Sequential(
#         Conv2d(in_channels = 256, out_channels = 512,
#                kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
#         BatchNorm2d(512),
#         self.activation(),
#         MaxPool2d(kernel_size = (2, 2), stride = (2, 2)),
# #         Conv2d(in_channels = 512, out_channels = 512,
# #                kernel_size = (3, 3), stride = (2, 2), padding = (1, 1)),
# #         BatchNorm2d(512),
# #         self.activation(),
# #         MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
#     )
 
#     self.fc_block_1 = Sequential(
#         Linear(in_features = 4 * 4 * 256, out_features = 2048),
#         self.activation(),
#         Dropout(0.3),
#         Linear(in_features = 2048, out_features = 512),
#         self.activation(),
#         Dropout(0.3),
#         Linear(in_features = 512, out_features = self.classes)
#     )
    
#     self.fc_block_2 = Sequential(
#         Linear(in_features = 4 * 4 * 512, out_features = 2048),
#         self.activation(),
#         Dropout(0.3),
#         Linear(in_features = 2048, out_features = 512),
#         self.activation(),
#         Dropout(0.3),
#         Linear(in_features = 512, out_features = self.classes)
#     )
    

#   def forward(self, x):
#     x = self.conv_block_1(x)
#     x = self.conv_block_2(x)
#     x = self.conv_block_3(x)
#     x = flatten(x, 1)
# #     x = self.fc_block_1(x)
#     x = self.fc_block_2(x)
#     out = LogSoftmax(x)

#     return x



In [10]:
num_classes = 10

# model = CNN(3, num_classes,nn.ReLU).to(device)  #Architecture 3
model = CNN(3, num_classes).to(device)  #Architecture 2
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 34, 34]           1,792
       BatchNorm2d-2           [-1, 64, 34, 34]             128
              ReLU-3           [-1, 64, 34, 34]               0
            Conv2d-4          [-1, 128, 36, 36]          73,856
       BatchNorm2d-5          [-1, 128, 36, 36]             256
              ReLU-6          [-1, 128, 36, 36]               0
            Conv2d-7          [-1, 128, 34, 34]         147,584
       BatchNorm2d-8          [-1, 128, 34, 34]             256
              ReLU-9          [-1, 128, 34, 34]               0
        MaxPool2d-10          [-1, 128, 17, 17]               0
           Conv2d-11          [-1, 256, 17, 17]         295,168
      BatchNorm2d-12          [-1, 256, 17, 17]             512
             ReLU-13          [-1, 256, 17, 17]               0
           Conv2d-14            [-1, 25

In [16]:
num_epochs = 20
learning_rate=0.001
momentum_value = 0.9
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum_value)

In [17]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            tepoch.set_postfix(loss=running_loss / ((tepoch.n + 1) * train_loader.batch_size))

    epoch_loss = running_loss / len(training_data)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/1563 [00:00<?, ?batch/s]

Epoch [1/20], Loss: 0.0043


  0%|          | 0/1563 [00:00<?, ?batch/s]

Epoch [2/20], Loss: 0.0046


  0%|          | 0/1563 [00:00<?, ?batch/s]

...

Epoch [18/20], Loss: 0.0030


  0%|          | 0/1563 [00:00<?, ?batch/s]

Epoch [19/20], Loss: 0.0029


  0%|          | 0/1563 [00:00<?, ?batch/s]

Epoch [20/20], Loss: 0.0027


In [18]:
loss_fn = torch.nn.CrossEntropyLoss()

def validate(model, dataloader, sanity_check=False):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []
    
    device = next(model.parameters()).device  
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0].to(device), batch[1].to(device) 
            
            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = loss_fn(output, y)
            total_loss += loss.detach().cpu().item() / len(dataloader)
            
            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())
        
    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')
    
    return total_loss, acc, f1, predictions


In [19]:
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)
test_loader

In [20]:
scores = validate(model,test_loader)
print(f"| Test Loss: {scores[0]: 7.3f}  | Test acc: {scores[1]: 1.5f}  | Test f1: {scores[2]: 1.5f}  |")

  0%|          | 0/313 [00:00<?, ?it/s]

| Test Loss:   1.642  | Test acc:  0.81980  | Test f1:  0.81939  |
